# Exercise sheet 2

Please write the names of all group members here:

Names: Vali Florinel Craciun, Javohir Isomurodov

## Exercise 1: Theoretical questions

## Exercise 1.1: 
    
Recall the basic cryptocurrency ScroogeCoin from the
lecure. In ScroogeCoin, suppose Mallory tries to generate (sk, pk) pairs until her
secret key matches someone else’s. What will she be able to do and how long will it
take before she succeeds on average? What if Alice’s random number generator has
a bug and her key generation procedure produces only 1,000 distinct pairs?

Generating $(sk,pk)$ pairs untill a match with an already existing $(sk,pk)$ pair is found can be translated in trying to find a collision, where two different rounds of `generateKeys(keysize)` lead to the same output. 
If we consider strings of 256-bits as keysize this means that, for the birthday paradox, that we need roughly $2^{\frac{256}{2}}$ hashes before getting a collision. 
If Alice's keys output space is reduced from $2^{256}$ to 1000, this means that Mallory will need only roughly $2^5$ trials, i.e 32 rounds. 
If Mallory is able to get Alice's sk she could forge Alice's signature and so sign documents on her behalf given that the `verify(pk,message,sig)` will always return True. 

## Exercise 1.2:

How does Bitcoin achieve consensus? How does consensus prevent double-spending? 

Consensus in Bitcoin is achieved in few steps:
- Each node collects a set of transactions to be presented as a block
- A random node is elected to propose his block
- Each other node verifies the content of the proposed block (coin is unspent, signatures are valid...)
- the voting happens: if the majority of nodes accept the block this gets confirmed and appended to the blockchain. His hash value and memory address will be included in the next block. 

This is what happens theoretically, but not practically. Real voting present some issues s.t.:
- vote multiplication by malicious nodes by creating multiple identities
- latency problems
- vote counting

A solution for this isssue is the so called "implicit consensus". In this setting the voting is made by simply including the suggested block's hash in the next block. If node majority does this, the probability of building on the proposed block is very high and the opposite happens if the majority is not reached. An important note to add is that implicit consensus works if the probability of picking an honest node leader is greather than 50%. 



Consensus prevent double spending by trying to get rid of branches in the blockchain as much as possible. 
Whenever a double spending attack happens both transactions are kept alive and, eventually, included in blocks. The blockchain is continued on the block received first and after some time the block containing the second transaction is added to the blockchain. This is where a branch starts because the 2 blocks referes to the same previous block. Since we need to get rid of branches, we need to decide which block to keep and which to abandon and this is done using confirmations. So, we initially allow double spending, untill one of the two blocks reaches at least k confirmations. When this happens the creator of the "good" block gets his reward, while the "bad" (also called orphan) block gets eliminated from the blockchain, leaving his creator without reward. 

## Exercise 2: Merkle Trees

Below you can find some useful imports 

In [5]:
# imports
import hashlib
import json
import ctypes

Below you can find some sample transactions with which you can build a Merkle tree. You can also use them to verify, whether or not your MerkleTree() class can handle an even and uneven number of leaves.

In [6]:
transactions_even = {
    't1': 'Alice sends 10 BTC to Bob',
    't2': 'Charlie sends 2.5 BTC to Bob',
    't3': 'Bob sends 5 BTC to Sarah',
    't4': 'Bob sends 4.5 BTC to Tim',
    't5': 'Sarah sends 1 BTC to Mary',
    't6': 'Paul sends 2 BTC to Jessica',
    't7': 'Jessica sends 1.5 BTC to Charlie',
    't8': 'Jessica sends 0.05 BTC to Carmen'
}

transactions_uneven = {
    't1': 'Alice sends 10 BTC to Bob',
    't2': 'Charlie sends 2.5 BTC to Bob',
    't3': 'Bob sends 5 BTC to Sarah',
    't4': 'Bob sends 4.5 BTC to Tim',
    't5': 'Sarah sends 1 BTC to Mary',
    't6': 'Paul sends 2 BTC to Jessica',
    't7': 'Jessica sends 1.5 BTC to Charlie',
    't8': 'Jessica sends 0.05 BTC to Carmen',
    't9' : 'Steffan sends 3.12 BTC to Michel'
}

It is up to you to decide if you input the transactions as dictionary or as a list

In [7]:
trans_even = list(transactions_even.values())
trans_odd = list(transactions_uneven.values())


### Exercise 2.1

Implement the class Node()

In [17]:
class Node():
    def __init__(self, leftHP, rightHP):
        '''
        Store the hash pointers to the two children of the node.
        leftHP: The hash pointer to the left child (either a leaf or another node)
        rightHP: The hash pointer to the right child (either a leaf or another node)
        '''
        self.leftHP = leftHP
        self.rightHP = rightHP
        self.leftChild = None
        self.rightChild = None
        self.data = None
    
    def get_left_child(self):
        '''
        The method returns the left child, the child is either a node or a leaf. The method does 
        not return the hash pointer to the child, but the entire node object or leaf
        leftHP: Hash pointer to the left child
        '''
        #self.leftChild = ctypes.cast(leftHP['pointer'], ctypes.py_object).value
        return self.leftChild
        
    
    def get_right_child(self):
        '''
        The method returns the right child, the child is either a node or a leaf. The method does 
        not return the hash pointer to the child, but the entire node object or leaf)
        rightHP: Hash pointer to the right child
        '''
        #self.rightChild = ctypes.cast(rightHP['pointer'], ctypes.py_object).value
        return self.rightChild
        
        

### Exercise 2.2

Implement the class MerkleTree()

In [44]:
class MerkleTree():
    def __init__(self):
        '''
        Store the Merkle tree and the Merkle root 
        '''
        self.tree = []
        self.Mroot = None
    
    def create_node(self, left, right):
        '''
        A node contains a hash pointers to each of two data_structures. These data structures are 
        either of type string (the transactions we use as input) or of type Node object.
        Use the class Node() to create nodes. The method returns a newly created node.
        
        left: left element to which the newly created node should store a hash pointer
        right: right element to which the newly created node should store a hash pointer
        '''
        
        leftHP={
            'pointer':id(left),
            'prev_hash':hash(left)
            }
        
        rightHP= {
            'pointer':id(right),
            'prev_hash':hash(right)
        }
        new_node = Node(leftHP,rightHP)

        # I decided to set here the right and left child. Maybe it's not super correct but made all the other methods work smoother
        new_node.leftChild=left 
        new_node.rightChild=right
        
        return new_node
        
        
    
    def parents_of_leaves(self, leaves: list):
        '''
        This method returns the parent nodes of the leaves.
        leaves: The leaves are stored in a list (or dictionary, this is up to you) and are of type 
                string.
        '''
        leaf_nodes=[] # transform everything in nodes
        for leaf in leaves:
            node=Node(None,None)
            node.data=leaf
            leaf_nodes.append(node)
        
        parents_ledger={}
        level=0
        while len(leaf_nodes)>1: # untill i reach the root
            new_parents = []
            level+=1

            for i in range(0,len(leaf_nodes),2):# work with pairs
                left=leaf_nodes[i]
                right=leaf_nodes[i+1] if (i+1) < len(leaf_nodes) else left # this is the condition for working also with odd trees 
                parent= self.create_node(left,right)
                new_parents.append(parent)
            
            
            leaf_nodes=new_parents
            parents_ledger[f'parents at level {level}'] = leaf_nodes

        self.Mroot= leaf_nodes[0]

        
        self.tree=leaf_nodes
        return parents_ledger
     
        
    def get_parents_from_node_objects(self, nodes: list):
        '''
        This method returns the parent nodes of nodes of type node object
        nodes: The nodes are stored in a list (or dictionary, this is up to you) and are of type node 
               object.
               
        Note: hashlib.sha256() unfortunately does not work for node objects. Alternatively you can use
              the hash() function from python. The output does not look as fancy and isn't as secure,
              but it should do the job of giving you a good idea of how Merkle trees work.
        '''
        parents_ledger={}
        level=0
        while len(nodes)>1: # untill i reach the root
            new_parents = []
            level+=1

            for i in range(0,len(nodes),2):# work with pairs
                left=nodes[i]
                right=nodes[i+1] if (i+1) < len(nodes) else left # this is the condition for also working with odd trees 
                parent= self.create_node(left,right)
                new_parents.append(parent)
            
            
            nodes=new_parents
            parents_ledger[f'parents at level {level}'] = nodes

        self.Mroot=nodes[0]
        self.tree=nodes
        return parents_ledger
        


    
    def get_root(self, data):
        '''
        This method returns the Merkle root of some data. The input data can therefore be considered as
        the leaves of the Merkle tree.
        data: The data which we use to build the Merkle tree and for which we return the Merkle root.
        '''
        self.parents_of_leaves(data)
        return self.Mroot
    
    def traverse_Mtree(self,Mroot):
    
        if Mroot==None:
            raise ValueError('Merkle tree doesn\'t exist yet')
        
        current_node=Mroot
        
        while True:
            leftC = current_node.get_left_child()
            if leftC==None:
                break 
            current_node=leftC

        first_leaf=current_node


        current_node=Mroot
        while True:
            rightC = current_node.get_right_child()
            
            if rightC==None:
                break
            
            current_node=rightC


        last_leaf=current_node

        return first_leaf.data,last_leaf.data

        

### Exercise 2.3

Use your newly implemented classes MerkleTree() and Node() to create a Merkle tree with the sample transactions at the top of this exercise sheet. Show, that your Merkle tree can handle an uneven number of inputs. Return the Merkle root.

In [45]:
merkletree=MerkleTree()

In [46]:
merkletree.parents_of_leaves(trans_even)

{'parents at level 1': [<__main__.Node at 0x1859006f2b0>,
 'parents at level 2': [<__main__.Node at 0x1859006c370>,
 'parents at level 3': [<__main__.Node at 0x1859006c2b0>]}

In [47]:
merkletree.parents_of_leaves(trans_odd)

{'parents at level 1': [<__main__.Node at 0x1859006c670>,
 'parents at level 2': [<__main__.Node at 0x1859006ca00>,
 'parents at level 3': [<__main__.Node at 0x1859006c790>,
 'parents at level 4': [<__main__.Node at 0x1859006c100>]}

In [48]:
merkletree.get_root(trans_odd)

Then use the Merkle root to go trough the tree and retrieve the left and rigth leaf. 


In [49]:
r=merkletree.Mroot

In [59]:
# I was not sure about what the exercise was precisely asking for. Right and left leaf of which node?
# In any case i put both the right and left node of the root and the leftmost and rightmost leaf in the tree.
# In the second case I returned directly the data inside the leaves (i.e. the transactions themselves, so 
# it's clearer that they're the first and last transaction of the odd list)
print(f'First interpretation: {r.get_left_child()} {r.get_right_child()}')
print('-----------------------')
print(f'Second interpretation: {merkletree.traverse_Mtree(r)}')


First interpretation: <__main__.Node object at 0x000001858FDF6200> <__main__.Node object at 0x000001858FDF5E70>
-----------------------
Second interpretation: ('Alice sends 10 BTC to Bob', 'Steffan sends 3.12 BTC to Michel')
